In [17]:
import json

from openai import OpenAI, ChatCompletion
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env

CLIENT = OpenAI()

In [4]:
with open('./assets/documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [18]:
def llm(
        prompt: str,
        client: OpenAI = CLIENT
) -> None:
    '''
    Gets a response from an llm using the user prompt provided.

    Args:
      prompt: A user prompt detailing the input to the llm.

      client: The OpenAI client which will be used to interface with the llm.
    '''
    response= client.chat.completions.create(
        messages=[{'role': 'user', 'content': prompt}],
        temperature=0.0,
        model='gpt-4o-mini',
        stream=True
    )

    for chunk in response:
      output = chunk.choices[0].delta.content
      print(output, end='', flush=True)

In [19]:
from elasticsearch import Elasticsearch

In [20]:
ES_CLIENT = Elasticsearch('http://localhost:9200') 

In [21]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions-new"

ES_CLIENT.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions-new/R8ANHv6vREyH8lfalKYXWQ] already exists')

In [11]:
from tqdm.auto import tqdm

In [12]:
for doc in tqdm(documents):
    ES_CLIENT.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [13]:
query = 'I just disovered the course. Can I still join it?'

In [14]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = ES_CLIENT.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [22]:
def es_rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    llm(prompt)

In [23]:
es_rag(query)

Yes, you can still join the course even after the start date. You are eligible to submit the homeworks, but be aware that there will be deadlines for turning in the final projects, so it's best not to leave everything until the last minute.None